In [1]:
import string
import numpy as np
from scipy.stats import norm
from statsmodels import robust
import pandas as pd
from IPython.display import display
pd.set_option('max_rows', 5)

## 外れ値・異常値・欠損値とは
---
<table class="text-left border">
    <tr>
        <th class="border-bottom border-right-bold">外れ値 (outlier)</th>
        <td>他の値から大きく外れた値</td>
        <td><li>特異なイベントの発生</li></td>
    </tr>
    <tr>
        <th class="border-bottom border-right-bold">異常値 (anomaly)</th>
        <td>データの生成プロセスに照らして、起こるべきでない異常によって生じた値<br />異常値である外れ値と異常値でない外れ値は<strong>データのみでは区別できない</strong></td>
        <td><li>センサーの故障</li><li>データの入力ミス</li><li>表計算ソフトの集計行混入</li></td>
    </tr>
    <tr>
        <th class="border-bottom border-right-bold">欠損値 (missing value)</th>
        <td>値の抜けているデータ</td>
        <td><li>NULL 値</li><li>空欄</li><li>プレースホルダー文字列</li></td>
    </tr>
</table>

外れ値・異常値の検出は中級編以降で扱い、ここでは欠損値の確認、検出した外れ値・欠損値の除去・補完を扱う。

## 欠損値の確認
---
`pandas.DataFrame.isnull`を使用する。

In [2]:
n_sample = 1000
n_features = 5
np.random.seed(1234)
sample = pd.DataFrame(
    np.random.normal(size=(n_sample, n_features)),
    columns=[string.ascii_uppercase[i] for i in range(n_features)])
for i, rate in enumerate(np.linspace(0, 0.5, n_features)):
    missing = np.random.choice(
        n_sample, size=int(n_sample * rate), replace=False)
    sample.iloc[missing, i] = np.nan

print('sample')
display(sample)

sample


,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


In [3]:
pd.DataFrame.isna??

In [4]:
sample.isna()

,A,B,C,D,E
0,False,False,False,True,False
1,False,False,False,False,True
...,...,...,...,...,...
998,False,False,True,False,True
999,False,False,False,False,False


列ごとに欠損値の有無を確かめるには`any`、欠損値の数を確かめるには`sum`を使用する。  
`pandas.DataFrame.info`を使用してもよい。

In [5]:
sample.isna().any()

A    False
B     True
C     True
D     True
E     True
dtype: bool

In [6]:
sample.isna().sum()

A      0
B    125
C    250
D    375
E    500
dtype: int64

In [7]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
A    1000 non-null float64
B    875 non-null float64
C    750 non-null float64
D    625 non-null float64
E    500 non-null float64
dtypes: float64(5)
memory usage: 39.2 KB


## 除去
---
最も単純な対処法だが、やりすぎるとデータが不足したり、必要な情報まで失われる可能性がある。  
欠損がランダムに発生するわけではない場合 (年齢の高い人や収入の高い人ほど収入を尋ねる欄に回答しないなど) は、**取り除いてしまうとデータに偏りが生じる**。

### 欠損値
---
`pandas.DataFrame.dropna`を使用する。

In [8]:
pd.DataFrame.dropna??

#### 欠損を含む行

In [9]:
sample.dropna()

,A,B,C,D,E
2,1.150036,0.991946,0.953324,-2.021255,-0.334077
7,-0.122092,0.124713,-0.322795,0.841675,2.390961
...,...,...,...,...,...
997,0.275159,0.075078,1.361621,-1.096261,-1.101649
999,-0.592877,0.708934,1.608824,1.995282,0.033269


#### 欠損を含む列

In [10]:
sample.dropna(axis=1)

,A
0,0.471435
1,0.887163
...,...
998,-0.442212
999,-0.592877


#### 全ての値が欠損している行

In [11]:
sample.loc[:, 'B':'E'].dropna(how='all')

,B,C,D,E
0,-1.190976,1.432707,NaN,-0.720589
1,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...
998,-1.350463,NaN,0.842840,NaN
999,0.708934,1.608824,1.995282,0.033269


#### 欠損でない値が任意の数以上ある行を残す

In [12]:
sample.dropna(thresh=3)

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


#### 特定の列に欠損がある行

In [13]:
sample.dropna(subset=['B'])

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


### 外れ値
---
条件式などを使って絞り込む。

#### 3$\sigma$法
---
平均から$\pm 3\sigma $ (標準偏差) を超える値を外れ値とする方法。正規分布では約 $0.03\%$ が外れ値になる。

In [14]:
mean = sample['A'].mean()
sigma = sample['A'].std(ddof=0)
sample.loc[(mean - 3 * sigma <= sample['A']) & (sample['A'] <= mean + 3 * sigma)]

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


###### 練習問題

正規分布で$\pm 3\sigma$範囲に含まれる値の割合を求める。

In [15]:
norm.cdf(3) - norm.cdf(-3)

0.9973002039367398

#### Hampel判別法 (Hampel identifier)
---
$3\sigma$法の平均を中央値に、標準偏差を中央絶対偏差 (Median Absolute Deviation) に置き換えた方法。$3\sigma$法よりサンプルに含まれる外れ値の影響を受けにくい。

$\displaystyle median( x) \pm 3\times ( MAD\times 1.4826)$

$\displaystyle MAD=median( |x_{i} -median( x) |)$

$1.4826$ は正規分布を仮定した場合に、中央絶対偏差 ($75\%$ 点に相当) を標準偏差に補正するための定数。

###### 練習問題

中央絶対偏差を求める以下の関数 mad を完成させる。

In [16]:
def mad(x):
    return

In [17]:
def mad(x):
    return np.median(np.abs(x - np.median(x)))

In [18]:
mad(sample['A'])

0.6558910693556668

中央絶対偏差は`statsmodels.robust.mad`で求められる。  
既定値では、正規化定数 $c=0.6744897501960817$ で割って補正してあるので、そのままの値を求めるには $c=1$ とする。 (正規化定数で割った値を中央絶対偏差と呼ぶこともある)

In [19]:
robust.mad??

In [20]:
robust.mad(sample['A'], c=1)

0.6558910693556668

###### 練習問題

標準正規分布の $75\%$ 点を求め、 $0.6744897501960817$ と比較する。  
$1$ を標準正規分布の $75\%$ 点で割り、 $1.4826$ と比較する。

In [21]:
print(norm.ppf(0.75))
print(1 / norm.ppf(0.75))

0.6744897501960817
1.482602218505602


###### 練習問題

Hampel 判別法に基づいて外れ値かどうかを判定する (外れ値でない場合に True 、外れ値の場合に False を返す) 以下の関数 Hampel を完成させる。

In [22]:
def hampel(ndarray):
    return

In [23]:
def hampel(ndarray):
    med = np.median(ndarray)
    mad = robust.mad(ndarray)
    return ((med - 3 * mad) <= ndarray) & (ndarray <= (med + 3 * mad))

In [24]:
sample.loc[hampel(sample['A'])]

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


#### 箱ひげ図
---
箱ひげ図では第 1 四分位点 ($Q_{1/4}$, lower quartile) 、第 3 四分位点 ($Q_{3/4}$, upper quartile) 、四分位範囲 (IQR, interquartile range) を用いて、以下のように外れ値を決めることが多い。

$Q_{1/4} -1.5*IQR,Q_{3/4} +1.5*IQR$

$IQR=Q_{3/4}-Q_{1/4}$

###### 練習問題

箱ひげ図における外れ値かどうかを判定する (外れ値でない場合に True 、外れ値の場合に False を返す) 以下の関数 isin_box を完成させる。

In [25]:
def isin_box(ndarray):
    return

In [26]:
def isin_box(ndarray):
    low = np.percentile(ndarray, 25)
    up = np.percentile(ndarray, 75)
    iqr = up - low
    return (low - 1.5 * iqr <= ndarray) & (ndarray <= up + 1.5 * iqr)

In [27]:
sample.loc[isin_box(sample['A'])]

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,NaN,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,NaN
...,...,...,...,...,...
998,-0.442212,-1.350463,NaN,0.842840,NaN
999,-0.592877,0.708934,1.608824,1.995282,0.033269


## 補完
---
欠損値を別の値で埋めること。  
ここでは代表値による補完を扱い、より高度な補完手法は中級編以降で扱う。

欠損値の補完には、`pandas.DataFrame.fillna`を使用する。

In [28]:
pd.DataFrame.fillna??

平均値での補完。

In [29]:
sample.fillna(sample.mean())

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,-0.018202,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,-0.004376
...,...,...,...,...,...
998,-0.442212,-1.350463,0.064951,0.842840,-0.004376
999,-0.592877,0.708934,1.608824,1.995282,0.033269


中央値での補完。

In [30]:
sample.fillna(sample.median())

,A,B,C,D,E
0,0.471435,-1.190976,1.432707,-0.045519,-0.720589
1,0.887163,0.859588,-0.636524,0.015696,0.019233
...,...,...,...,...,...
998,-0.442212,-1.350463,0.050845,0.842840,0.019233
999,-0.592877,0.708934,1.608824,1.995282,0.033269
